# This notebook trains NBEATS model on the first cluster buildings. labeled-dataset contains the data of all the buildings in the first cluster

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Function to optimize data types
def optimize_data_types(df):
    # Optimize numeric columns
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = df[col].astype('int32')
    
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].astype('float32')

    # Optimize object columns
    for col in df.select_dtypes(include=['object']).columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')
    
    return df

# Load the data and remove the first and last columns
df = pd.read_csv('/kaggle/input/labeled-dataset/df_labeled0.csv')
df = df.iloc[:, 1:-1]

# Optimize data types
df = optimize_data_types(df)

# Verify DataFrame shape
print("DataFrame shape:", df.shape)
print("DataFrame head:", df.head())

# Define the window size for lagged inputs
window_size = 40  # Replace N with your desired window size

# Function to process each chunk
def process_chunk(chunk, window_size):
    data = []
    for i, row in chunk.iterrows():
        values = row.values
        for t in range(window_size, len(values)):
            lagged_inputs = values[t-window_size:t]
            target_value = values[t]

            timestamp = pd.Timestamp('2009-07-14 00:00:00') + pd.Timedelta(hours=t)
            year = timestamp.year
            month = timestamp.month
            day = timestamp.day
            hour = timestamp.hour

            data.append(list(lagged_inputs) + [year, month, day, hour, target_value])
    return data

# Create an empty DataFrame to store results
columns = [f'lag_{i}' for i in range(1, window_size+1)] + ['year', 'month', 'day', 'hour', 'target']
df_transformed = pd.DataFrame(columns=columns)

# Process the data in chunks
chunk_size = 100  # Adjust based on available memory
for start_row in range(0, df.shape[0], chunk_size):
    chunk = df.iloc[start_row:start_row + chunk_size]
    data = process_chunk(chunk, window_size)
    df_chunk_transformed = pd.DataFrame(data, columns=columns)
    df_transformed = pd.concat([df_transformed, df_chunk_transformed], ignore_index=True)


ordinal_encoder = OrdinalEncoder()
df_transformed[['month', 'day', 'hour']] = ordinal_encoder.fit_transform(df_transformed[['month', 'day', 'hour']])


print("Transformation complete. The new dataset is saved as 'transformed_dataset.csv'.")



In [ ]:
######CORRECT IT
import pandas as pd
import gc
# Convert the date columns to datetime
df_transformed['year1'] = df_transformed['year']
df_transformed['month1'] = df_transformed['month']
df_transformed['day1'] = df_transformed['day']
df_transformed['hour1'] = df_transformed['hour']
df_transformed["target1"] = df_transformed["target"]

# Define the date ranges for training and testing
train_start_date = '2009-07-14'
train_end_date = '2010-12-15'
test_start_date = '2010-12-15'
test_end_date = '2011-01-01'

# Convert the date columns to datetime
df_transformed['year'] = df_transformed['year'].astype(int)
df_transformed['month'] = df_transformed['month'].astype(int)+1
df_transformed['day'] = df_transformed['day'].astype(int)+1
df_transformed['hour'] = df_transformed['hour'].astype(int)+1

# Create a datetime column
df_transformed['date_time'] = pd.to_datetime(df_transformed[['year', 'month', 'day', 'hour']])
df_sample = df_transformed.loc[:12825]

# Set the datetime column as the index
df_transformed.set_index('date_time', inplace=True)
df_sample.set_index('date_time', inplace=True)

# Drop the irrelevant columns
df_transformed.drop(columns=['year', 'month', 'day', 'hour','target'], inplace=True)
df_sample.drop(columns=['year', 'month', 'day', 'hour','target'], inplace=True)

df_transformed = df_transformed.sort_index()
df_sample = df_sample.sort_index()

# Split the data into training and testing sets
train = df_transformed.loc[train_start_date:train_end_date]

test = df_transformed.loc[test_start_date:test_end_date]


# Create sample data for the first building
sample = df_sample.loc[test_start_date:test_end_date]



del df_transformed, df
gc.collect()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchmetrics import MeanSquaredError, SymmetricMeanAbsolutePercentageError
from torch.utils.data import DataLoader, Dataset
import time
from memory_profiler import memory_usage
import resource
import pickle
import gc
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


# Define a function to create the model for memory profiling
def create_model():
    model = CombinedModel(nbits_params, embedding_dim, final_hidden)
    return model

def get_memory_usage():
    # Return current memory usage in MB
    return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

class TrendBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(TrendBlock, self).__init__()
        self.fc = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.fc:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class SeasonalityBlock(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers):
        super(SeasonalityBlock, self).__init__()
        self.fc = nn.ModuleList([nn.Linear(input_size, num_hidden)] +
                                [nn.Linear(num_hidden, num_hidden) for _ in range(num_layers - 1)])
        self.backcast_fc = nn.Linear(num_hidden, input_size)
        self.forecast_fc = nn.Linear(num_hidden, output_size)

    def forward(self, x):
        for layer in self.fc:
            x = torch.relu(layer(x))
        backcast = self.backcast_fc(x)
        forecast = self.forecast_fc(x)
        return backcast, forecast

class NBEATS(nn.Module):
    def __init__(self, input_size, output_size, num_hidden, num_layers, num_blocks):
        super(NBEATS, self).__init__()
        self.trend_blocks = nn.ModuleList([TrendBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks // 2)])
        self.seasonality_blocks = nn.ModuleList([SeasonalityBlock(input_size, output_size, num_hidden, num_layers) for _ in range(num_blocks // 2)])

    def forward(self, x):
        backcast, forecast = x, 0
        for block in self.trend_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        for block in self.seasonality_blocks:
            backcast, block_forecast = block(x)
            x = x - backcast
            forecast = forecast + block_forecast
        return forecast

class EmbeddingNetwork(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EmbeddingNetwork, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

class YearNetwork(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super(YearNetwork, self).__init__()
        self.hidden = nn.Linear(input_dim, 25)
        self.output = nn.Linear(25, num_hidden)

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        return self.output(x)

class CombinedModel(nn.Module):
    def __init__(self, nbits_params, embedding_dim, final_hidden):
        super(CombinedModel, self).__init__()
        self.nbeats = NBEATS(**nbits_params)
        self.month_net = EmbeddingNetwork(12, embedding_dim)  # Months from 1 to 12
        self.day_net = EmbeddingNetwork(31, embedding_dim)    # Days from 1 to 31
        self.hour_net = EmbeddingNetwork(24, embedding_dim)   # Hours from 0 to 23
        self.year_net = YearNetwork(1, 10)
        self.final_layer = nn.Sequential(
            nn.Linear(embedding_dim * 3 + nbits_params['output_size'] + 10, final_hidden),
            nn.ReLU(),
            nn.Linear(final_hidden, 1)
        )

    def forward(self, x, month, day, hour, year):
        ts_output = self.nbeats(x)
        month_output = self.month_net(month.long()).squeeze(1)
        day_output = self.day_net(day.long()).squeeze(1)
        hour_output = self.hour_net(hour.long()).squeeze(1)
        year_output = self.year_net(year.float())
        combined_output = torch.cat((ts_output, month_output, day_output, hour_output, year_output), dim=1)
        final_output = self.final_layer(combined_output)
        return final_output


# Example usage:
input_size = 40  # Length of input time series
output_size = 1  # Length of output time series (forecast)
num_blocks = 12
num_hidden = 512
num_layers = 8
embedding_dim = 10
final_hidden = 256

nbits_params = {
    'input_size': input_size,
    'output_size': output_size,
    'num_blocks': num_blocks,
    'num_hidden': num_hidden,
    'num_layers': num_layers
}

start_mem = get_memory_usage()
net = create_model()
end_mem = get_memory_usage()
print(f"Memory used for model creation: {end_mem - start_mem} MB")

device = "cuda"
net.to(device)

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx].astype("float32")
        features = torch.FloatTensor(row[:-5].values)  # All columns except the last 5
        year = torch.FloatTensor([row[-5]]).to(torch.int)
        month = torch.FloatTensor([row[-4]]).to(torch.int)  # Month is already 0-indexed by OrdinalEncoder
        day = torch.FloatTensor([row[-3]]).to(torch.int)    # Day is already 0-indexed by OrdinalEncoder
        hour = torch.FloatTensor([row[-2]]).to(torch.int)   # Hour is already 0-indexed by OrdinalEncoder
        label = torch.FloatTensor([row[-1]])  # The last column
        return features, year, month, day, hour, label

    def __len__(self):
        return len(self.dataframe)

def create_data_loaders(data, chunk_size, batch_size):
    chunks = np.array_split(data, len(data) // chunk_size)
    data_loaders = []
    for chunk in chunks:
        dataset = CustomDataset(chunk)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        data_loaders.append(data_loader)
    return data_loaders


lr = 0.0005
n_epochs = 10
window_size = 40
chunk_size = 122880
batch_size = 1024*12*5

# Assuming `train`, `test`, `sample` are pre-loaded DataFrames
train_loaders = create_data_loaders(train, chunk_size, batch_size)
test_loaders = create_data_loaders(test, chunk_size, batch_size)
sample_loaders = create_data_loaders(sample, 100, 1)
print("data loaders are ready")

del train, test, sample
gc.collect()

def train_function(net, criterion, optimizer, data_loaders, n_epochs=5, device=torch.device("cpu")):
    from torch.optim.lr_scheduler import ReduceLROnPlateau
    scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, threshold=0.1, patience=3, factor=0.5)
    for epoch in range(n_epochs):
        epoch_loss = 0
        counter = 0
        for data_loader in data_loaders:
            for features, year, month, day, hour, labels in data_loader:
                features = features.to(device)
                year = year.to(device)
                month = month.to(device)
                day = day.to(device)
                hour = hour.to(device)
                labels = labels.to(device)

                outputs = net(features, month, day, hour, year)
                loss = criterion(outputs, labels)
                epoch_loss += loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                print("c bon")
        scheduler.step(epoch_loss / len(data_loaders))
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(data_loaders)}")
        with open(f'modelcheckpoint{epoch}.pickle', 'wb') as handle:
            pickle.dump([net, optimizer], handle, protocol=pickle.HIGHEST_PROTOCOL)
        collected = gc.collect()
    return net

def test_function(net, data_loaders, scaler, label_scaler, device=torch.device("cuda"), return_data=False):
    mse = MeanSquaredError().to(device)
    smape = SymmetricMeanAbsolutePercentageError().to(device)
    net.eval()
    list_outputs = []
    list_targets = []
    with torch.no_grad():  # to not reserve a memory space for gradients
        for data_loader in data_loaders:
            for features, year, month, day, hour, labels in data_loader:
                features = features.to(device)
                year = year.to(device)
                month = month.to(device)
                day = day.to(device)
                hour = hour.to(device)
                labels = labels.to(device)

                outputs = net(features, month, day, hour, year)
                outputs = outputs.squeeze()
                labels = labels.squeeze()

                mse(outputs, labels)
                smape(outputs, labels)
                list_targets.append(labels.detach().unsqueeze(0))  # Ensure tensors are at least one-dimensional
                list_outputs.append(outputs.detach().unsqueeze(0))  # Ensure tensors are at least one-dimensional
    test_mse = mse.compute()
    test_smape = smape.compute()
    print(f"Test MSE: {test_mse} , SMAPE {test_smape}")
    if return_data:
        return torch.cat(list_outputs), torch.cat(list_targets), test_mse

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

net.to(device)

print("Model Started")

start_mem = get_memory_usage()
time1 = time.time()
net = train_function(net, criterion, optimizer, train_loaders, n_epochs=n_epochs, device=torch.device(device))
time2 = time.time()
print("training time is ", time2 - time1)
end_mem = get_memory_usage()
print(f"Memory used for model training: {end_mem - start_mem} MB")

net.to(device)
time3 = time.time()
test_function(net, test_loaders, None, None, torch.device(device))
time4 = time.time()
print("inference time is ", time4 - time3)

sample_outputs, sample_targets, sample_mse = test_function(net, sample_loaders, None, None, torch.device(device), True)
plt.plot(sample_outputs.to("cpu"), "-o", color="blue", label="NBEATS Predictions", markersize=3)
plt.plot(sample_targets.to("cpu"), color="red", label="Actual")
plt.ylabel("Energy Consumption (MW)")
plt.title(f"Energy Consumption for Electricity state 1st building")
plt.legend()
plt.show()
